<a href="https://colab.research.google.com/github/Ayelets/Mysterious-Disappearances/blob/master/Baot_workshop_Fine_tuning_Stable_Diffusion_with_DreamBooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stable Diffusion

## Initial setup

In [ ]:
!pip install bitsandbytes transformers accelerate peft -q
!pip install git+https://github.com/huggingface/diffusers.git -q
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
--2024-06-21 07:38:40--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84303 (82K) [text/plain]
Saving to: ‘train_dreambooth_lora_sdxl.py’

train_dreambooth_lo 100%[===================>]  82

In [ ]:
#@title Import required libraries
import argparse
import itertools
import math
import os
import random

import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.utils.data import Dataset

import PIL
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from diffusers import AutoencoderKL, DDPMScheduler, PNDMScheduler, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer


/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols
    imgs = [img.resize((512, 512)) for img in imgs]
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-du50veo5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-du50veo5
  Resolved https://github.com/huggingface/accelerate to commit 1f7a79b428749f45187ec69485f2c966fe21926e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.32.0.dev0-py3-none-any.whl size=311882 sha256=12341e957546a87ff066f9c152050c9e6b7d9c17b81f3aa57228241417067529
  Stored in directory: /tmp/pip-ephem-wheel-cache-h88n05tb/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.31.0
    Uninstalling accelerate-0.31.0:
      Successfully uninstalled accelerate-0.31.0


In [ ]:
from diffusers import DiffusionPipeline
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16",low_cpu_mem_usage=False)
pipe.to("cuda")
f'{torch.cuda.memory_stats()["reserved_bytes.all.current"] / 1024 / 1024} MB'


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

text_encoder_2/config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

after


'6946.0 MB'

In [ ]:
num_images = 3 #@param {type:"number"}
prompt = "A painting of Lassie dog in style of Vincent Van Gogh" #@param {type:"string"}
num_inference_steps = 20 #@param {type:"number"}
guidance_scale=7.5#@param {type:"number"}
prompts=[prompt]* num_images
generator = [torch.Generator(device="cuda").manual_seed(i*15) for i in range(num_images)]
images = pipe(prompts,num_inference_steps = num_inference_steps, generator=generator, guidance_scale=guidance_scale).images
grid = image_grid(images, rows=1, cols = num_images)
grid

prompts examples:https://civitai.com/

In [ ]:
prompt = "score_9, score_8_up, score_7_up, BREAK scenery, landscape, tile wall, fog, grass, window, building, stucco, path, outdoors, tree, east asian architecture, no humans, dirt, autumn leaves, architecture, blue sky, bush, castle, plant, pagoda, sky, wall, stairs, town, nature, cloudy sky, mountain, light, green theme, cloud, stone wall, entrance" #@param {type:"string"}
negative_prompt="realistic, greyscale, monochrome"#@param {type:"string"}
num_inference_steps = 20 #@param {type:"number"}
guidance_scale=7#@param {type:"number"}
prompts=[prompt]
#generator = [torch.Generator(device="cuda").manual_seed(i*15) for i in range(num_images)]
generator=[torch.Generator(device="cuda").manual_seed(2856396049)]
images = pipe(prompts,negative_prompt=negative_prompt,num_inference_steps = num_inference_steps, generator=generator, guidance_scale=guidance_scale).images
images[0]

Delete the model to free up space

In [ ]:
import gc
import time

def free_pipe(pipe):
  del pipe
  time.sleep(2)
  gc.collect()
  time.sleep(2)
  torch.cuda.empty_cache()
  time.sleep(2)
  f'after garbage collection: {torch.cuda.memory_stats()["reserved_bytes.all.current"] / 1024 / 1024} MB'

free_pipe(pipe)

after garbage collection


'20.0 MB'

Based on the [notebook](https://github.com/huggingface/notebooks/blob/main/diffusers/SDXL_DreamBooth_LoRA_.ipynb)





#**Settings for teaching your new concept**

In [ ]:
import os
from google.colab import files


# pick a name for the image folder
local_dir = "./yana/" #@param
os.makedirs(local_dir)
os.chdir(local_dir)

# choose and upload local images into the newly created directory
uploaded_images = files.upload()
os.chdir("/content") # back to parent directory

Saving yana(2).JPG to yana(2).JPG
Saving yana(3).JPG to yana(3).JPG
Saving yana(4).JPG to yana(4).JPG
Saving yana(5).JPG to yana(5).JPG
Saving yana(6).JPG to yana(6).JPG
Saving yana(7).JPG to yana(7).JPG
Saving yana(8).JPG to yana(8).JPG
Saving yana(9).JPG to yana(9).JPG
Saving yana(10).JPG to yana(10).JPG
Saving yana(11).JPG to yana(11).JPG
Saving yana(12).JPG to yana(12).JPG
Saving yana(13).JPG to yana(13).JPG
Saving yana.JPG to yana.JPG


In [ ]:
import glob
from PIL import Image
img_paths = f"{local_dir}*"
imgs = [Image.open(path).convert("RGB").resize((1024, 1024)) for path in glob.glob(img_paths)]
imgs_and_paths = [(path,Image.open(path).convert("RGB").resize((1024, 1024))) for path in glob.glob(img_paths)]
image_grid(imgs, 1, len(imgs))

#**Generate custom captions with BLIP**


In [ ]:
import requests
from transformers import AutoProcessor, BlipForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# load the processor and the captioning model
blip_processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base",torch_dtype=torch.float16).to(device)

# captioning utility
def caption_images(input_image):
    inputs = blip_processor(images=input_image, return_tensors="pt").to(device, torch.float16)
    pixel_values = inputs.pixel_values

    generated_ids = blip_model.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_caption

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
for img in imgs_and_paths:
  print(caption_images(img[1]))

a painting of people playing instruments in a city
a painting of people walking down a street
a woman holding a child in her arms
two paintings of people in a green field
a painting of a city with people and buildings
a painting of people drinking wine and talking
a painting of a city with a tower
a painting of people walking around a market
a painting of a man and woman in a village
a painting of people in a city
a painting of two women sitting on a bench
a painting of a lion on a yellow background
a painting of a woman in a green field


If there are too specific words in the image caption that describe your concept such as the animal species or its color. write them here with "," separates between them. (e.g black,Maine Coon).
A description that is too informative may cause the new concept not to be learned properly, so we would prefer to remove them.


In [ ]:
too_specific_descript="" #@param {type:"string"}
words_for_removal = too_specific_descript.split(",")

Now let's add the concept token identifier (e.g. TOK) to each caption using a caption prefix. Feel free to change the prefix according to the concept you're training on!

for this example we can use "a photo of TOK," other options include:
For styles - "In the style of TOK"
For faces - "photo of a TOK person"
You can add additional identifiers to the prefix that can help steer the model in the right direction.
e.g. for this example, instead of "a photo of TOK" we can use "a photo of TOK dog" / "a photo of TOK corgi dog"

In [ ]:
import json
caption_prefix = "In the style of TOK" #@param {type:"string"}
with open(f'{local_dir}metadata.jsonl', 'w') as outfile:
  for img in imgs_and_paths:
      img_automatic_caption=caption_images(img[1]).split("\n")[0]
      for word in words_for_removal:
        img_automatic_caption=img_automatic_caption.replace(word, '').replace("  ", ' ')
      caption = caption_prefix +" "+ img_automatic_caption
      entry = {"file_name":img[0].split("/")[-1], "prompt": caption}
      json.dump(entry, outfile)
      outfile.write('\n')

Free some memory:

In [ ]:
import gc

# delete the BLIP pipelines and free up some memory
del blip_processor, blip_model
time.sleep(2)
gc.collect()
torch.cuda.empty_cache()

## Prep for training 💻

Initialize `accelerate`:

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


### Log into your Hugging Face account
Pass [your **write** access token](https://huggingface.co/settings/tokens) so that we can push the trained checkpoints to the Hugging Face Hub:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Train! 🔬

#### Set Hyperparameters ⚡
To ensure we can DreamBooth with LoRA on a heavy pipeline like Stable Diffusion XL, we're using:

* Gradient checkpointing (`--gradient_accumulation_steps`)
* 8-bit Adam (`--use_8bit_adam`)
* Mixed-precision training (`--mixed-precision="fp16"`)

### Launch training 🚀🚀🚀

To allow for custom captions we need to install the `datasets` library, you can skip that if you want to train solely
 with `--instance_prompt`.
In that case, specify `--instance_data_dir` instead of `--dataset_name`

In [ ]:
!pip install datasets -q --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.8 MB/s eta 0:00:00
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you'll have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you'll have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you'll have pyarrow 16.1.0 which is incompatible.


 - Use `--output_dir` to specify your LoRA model repository name!
 - Use `--caption_column` to specify name of the cpation column in your dataset. In this example we used "prompt" to
 save our captions in the
 metadata file, change this according to your needs.

In [ ]:
from huggingface_hub import whoami
from pathlib import Path
#@markdown make sure the `output_dir` you specify here is the same as the one used for training
output_dir = "yana_style" #@param{type:"string"}
username = whoami(token=Path("/root/.cache/huggingface/"))["name"]
repo_id = f"{username}/{output_dir}"

In [ ]:
dataset_name = local_dir
output_dir = output_dir
instance_prompt = caption_prefix

!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --dataset_name="{dataset_name}" \
  --output_dir="{output_dir}" \
  --caption_column="prompt"\
  --mixed_precision="fp16" \
  --instance_prompt="{instance_prompt}" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=3 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --snr_gamma=5.0 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --max_train_steps=500 \
  --checkpointing_steps=717 \
  --seed="0"

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
2024-06-20 16:14:42.461221: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 16:14:42.461304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 16:14:42.462916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] 

### Save your model to the hub and check it out 🔥

In [ ]:
# @markdown Sometimes training finishes succesfuly (i.e. a **.safetensores** file with the LoRA weights saved properly to your local `output_dir`) but there's not enough RAM in the free tier to push the model to the hub 🙁
# @markdown
# @markdown To mitigate this, run this cell with your training arguments to make sure your model is uploaded! 🤗

# push to the hub🔥
from train_dreambooth_lora_sdxl import save_model_card
from huggingface_hub import upload_folder, create_repo

repo_id = create_repo(repo_id, exist_ok=True).repo_id

# change the params below according to your training arguments
save_model_card(
    repo_id = repo_id,
    images=[],
    base_model="stabilityai/stable-diffusion-xl-base-1.0",
    train_text_encoder=False,
    instance_prompt = caption_prefix,
    validation_prompt=None,
    repo_folder=output_dir,
    vae_path="madebyollin/sdxl-vae-fp16-fix",
    use_dora=False
)

upload_folder(
    repo_id=repo_id,
    folder_path=output_dir,
    commit_message="End of training",
    ignore_patterns=["step_*", "epoch_*"],
)

README.md:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

events.out.tfevents.1718900144.4ecee80347ce.5226.1:   0%|          | 0.00/3.30k [00:00<?, ?B/s]

pytorch_lora_weights.safetensors:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

events.out.tfevents.1718900144.4ecee80347ce.5226.0:   0%|          | 0.00/109k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ayelets/yana_style/commit/97310e3babc41c9c942ce50a8efa8deff169a400', commit_message='End of training', commit_description='', oid='97310e3babc41c9c942ce50a8efa8deff169a400', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from IPython.display import display, Markdown

link_to_model = f"https://huggingface.co/{repo_id}"
display(Markdown("### Your model has finished training.\nAccess it here: {}".format(link_to_model)))

### Your model has finished training.
Access it here: https://huggingface.co/ayelets/yana_style

Let's generate some images with it!

## Inference 🐕

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    low_cpu_mem_usage=False
)

pipe.load_lora_weights(repo_id)
_ = pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_lora_weights.safetensors:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

In [ ]:
#@title Run the Stable Diffusion pipeline
#@markdown Don't forget to use the placeholder token in your prompt

prompt = "A painting of a cat drinks milk style of TOK In the style of TOK" #@param {type:"string"}
negative_prompt="photo realistic "#@param {type:"string"}
num_samples = 3 #@param {type:"number"}


all_images = []
num_rows=1

images = pipe([prompt] * num_samples, negative_prompt=negative_prompt, num_inference_steps=30, guidance_scale=7.5).images
all_images.extend(images)

grid = image_grid(all_images, num_rows, num_samples)
grid

#delete model

In [ ]:
free_pipe(pipe)

#Load a trained model

In [ ]:
from diffusers import DiffusionPipeline
model_path="ayelets/lager_dog_LoRA"#@param {type:"string"}
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16",low_cpu_mem_usage=False)
pipe.load_lora_weights(model_path)
pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_lora_weights.safetensors:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.30.0.dev0",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
prompt = "A painting of a mashup of TOK dog and a bunny" #@param {type:"string"}
negative_prompt=""#@param {type:"string"}
num_samples = 2 #@param {type:"number"}

all_images = []
num_rows=1

images = pipe([prompt] * num_samples, negative_prompt=negative_prompt, num_inference_steps=30, guidance_scale=7.5).images
all_images.extend(images)

grid = image_grid(all_images, num_rows, num_samples)
grid